In [1]:
import csv
import copy
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import graph_tool as gt
import graph_tool.util as gu
import os
import io
import base64
import pandas as pd

from dash import Dash, dcc, html, Input, Output, State, ctx
from graph_tool import GraphView
from jupyter_dash import JupyterDash

In [2]:
def parse_file_to_network(df, g, sample):
    edge_d = df.set_index('REM').T.to_dict('list')
    nodes = []
    edges= []
    for re, ge in edge_d.items():
        v_list = gu.find_vertex(g, g.vp.name, re)
        v = v_list[0]
        if g.vp.name[v].startswith("E") and sample != "score" and sample != "p_value":
            nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'expr': str(g.vp[sample][v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
        else: nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
        for e in v.all_edges():
            v2 = e.target()
            if g.vp.name[v2] == ge[0].strip():
                if g.edge_properties[sample][e] >= 0: color = "green"
                else: color = "red"
                edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': str(g.edge_properties[sample][e]), 'rem': g.ep.rem[e]}, 'style':{'width':abs(g.edge_properties[sample][e])*5, 'line-color': color}})
                if g.vp.name[v2].startswith("E") and sample != "score" and sample != "p_value":
                    nodes.append({'data': {'id': g.vp.name[v2], 'label': g.vp.name[v2], 'start': str(g.vp.start[v2]), 'end': str(g.vp.end[v2]), 'expr': str(g.vp[sample][v2]), 'chr': g.vp.chr[v2]}, 'classes':'DodgerBlue'})
                else:
                    nodes.append({'data': {'id': g.vp.name[v2], 'label': g.vp.name[v2], 'start': str(g.vp.start[v2]), 'end': str(g.vp.end[v2]), 'chr': g.vp.chr[v2]}, 'classes': 'DodgerBlue'})
    cyto_list = edges+nodes
    return cyto_list


In [3]:
def get_network(node, sample, fileupload = False, df = None):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    g = gr
    if fileupload is True:
        return parse_file_to_network(df, g, sample)
    v_list = gu.find_vertex(g, g.vp.name, node)
    if len(v_list) != 0:
        v = v_list[0]
    else:
        ed = gu.find_edge(g, g.ep.rem, node)[0]
        v = ed.source()

    nodes = []
    edges= []
    if g.vp.name[v].startswith("E") and sample != "score" and sample != "p_value":
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'expr': str(g.vp[sample][v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
    else:
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
    for e, w in zip(v.all_edges(), v.all_neighbors()):
            if g.edge_properties[sample][e] >= 0: color = "green"
            else: color = "red"
            edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': str(g.edge_properties[sample][e]), 'rem': g.ep.rem[e]}, 'style':{'width':abs(g.edge_properties[sample][e])*5, 'line-color': color}})
            if g.vp.name[w].startswith("E") and sample != "score" and sample != "p_value":
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': str(g.vp.start[w]), 'end': str(g.vp.end[w]), 'expr': str(g.vp[sample][w]), 'chr': g.vp.chr[w]}, 'classes':'DodgerBlue'})
            else:
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': str(g.vp.start[w]), 'end': str(g.vp.end[w]), 'chr': g.vp.chr[w]}, 'classes': 'DodgerBlue'})
    cyto_list = edges+nodes
    return cyto_list

In [4]:
def filter_network(graph, weight_filter_up, weight_filter_low, checklist_show):
    netw = copy.deepcopy(graph)
    filt_net = []
    filt_nodes = set()
    if netw is not None:
        for e in netw:
            if e.get('data').get('source') is not None:
                if not ((weight_filter_up != None and float(e.get('data').get('label')) > weight_filter_up) or (weight_filter_low != None and float(e.get('data').get('label')) < weight_filter_low)):
                    if 'Show weights' not in checklist_show:
                        e.get('data').pop('label')
                        filt_net.append(e)
                    else:
                        filt_net.append(e)
                    filt_nodes.add(e.get('data').get('source'))
                    filt_nodes.add(e.get('data').get('target'))
            elif e.get('data').get('id') in filt_nodes:
                filt_net.append(e)
    return filt_net

In [6]:
app = Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Div([
        html.H1("REMnet", style={"textAlign":"center", "text-decoration":"underline", "fontSize":40, 'font-family':'Consolas'})
    ]),

    dcc.Loading(
        children=[
            html.Div([
                html.Div([
                    html.Div([
                        html.Div([
                            html.Div([
                                # html.H3("Detailed Analysis", style={"text-decoration":"underline", "fontSize":20, 'font-family':'Consolas'}),
                                "Select celltype",
                                dcc.Dropdown(
                                    id='dropdown-graph',
                                    value=os.listdir('../data/graph-tool/')[0][:-3],
                                    clearable=False,
                                    options=[
                                        {'label': name[:-3], 'value': name[:-3]}
                                        for name in os.listdir('../data/graph-tool/')
                                    ]
                                )
                            ], style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                "Select sample",
                                dcc.Dropdown(
                                    id= 'dropdown-sample',
                                    value= '',
                                    clearable=False,
                                    options=[]
                                )
                            ], style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                "Select gene",
                                dcc.Dropdown(
                                id='dropdown-gen',
                                clearable=False,
                                options=[]
                                )
                            ],style={'display': 'block', 'height': '80px'}
                            ),
                            html.Div([
                                html.P('Enter a CREM ID from 1 to 365.286: '),
                                dcc.Input(id='input-crem', type='number', value = 1, step=1, min=1, max=365286, debounce=True),
                            ], style={'height': '80px'}
                            ),
                            html.Div([
                                html.P('Enter a REM ID from 1 to 2.404.862: '),
                                dcc.Input(id='input-rem', type='number', value = 1, step=1, min=1, max=2404862, debounce=True),
                            ], style={'height': '80px'}
                            ),
                        ], style={'display': 'inline-block', 'padding-left' : '10px', 'padding-right' : '10px', 'padding-top' : '10px', 'padding-bottom' : '10px'}),
                    ], style={"border":"1px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

                    html.Div([
                        html.Div([
                            html.Div([
                                html.P("Enter gene degree filter: "),
                                dcc.Slider(id='filter-gen', min=0, max=108, step=1, value=0, marks=None, tooltip={"placement": "bottom", "always_visible": True}),
                            ], style={'height': '80px'}
                            ),

                            html.Div([
                                "Set an upper bond weight filter:  ",
                                dcc.Input(id='filter-weight-up', type='number', debounce=True),
                                html.Div(id= 'filter-text-up')
                            ], style={'height': '80px'}),
                            html.Div([
                                "Set a lower bond weight filter:  ",
                                dcc.Input(id='filter-weight-low', type='number', debounce=True),
                                html.Div(id= 'filter-text-low')
                            ], style={'height': '80px'}),
                        ], style={'display': 'inline-block', 'padding-left' : '10px', 'padding-right' : '10px', 'padding-top' : '10px', 'padding-bottom' : '10px'}),
                    ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

                    dbc.Button('Reset filter', id= 'reset-filter', n_clicks=0, outline=True, color="primary", className="me-1")

                ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '14%'}
                ),
                html.Div([
                    html.Div([
                        cyto.Cytoscape(
                            id='network-graph',
                            style={'width': '100%', 'height': '700px'},
                            layout={'name': 'circle', 'avoidOverlap': 'true', 'nodeDimensionsIncludeLabels': 'true'},
                            elements=[],
                            stylesheet=[
                    #             # Group selectors
                                {
                                    'selector': 'node',
                                    'style': {
                                        'content': 'data(label)'
                                    }
                                },
                                {
                                    'selector': 'edge',
                                    'style': {
                                        'curve-style': 'bezier',
                                        'content': 'data(label)'
                                    }
                                },
                                {
                                    'selector': '.DodgerBlue',
                                    'style': {
                                        'background-color': 'DodgerBlue',
                                        'line-color': 'DodgerBlue'
                                    }
                                },
                                {
                                    'selector': '.black',
                                    'style': {
                                        'background-color': 'black',
                                        'line-color': 'black'
                                    }
                                },
                                {
                                    'selector': '.green',
                                    'style': {
                                        'background-color': 'green',
                                        'line-color': 'green'
                                    }
                                },
                                {
                                    'selector': '.red',
                                    'style': {
                                        'background-color': 'red',
                                        'line-color': 'red'
                                    }
                                }
                            ]
                        ),
                    ]),
                ], style={ 'vertical-align': 'top', 'display': 'inline-block', 'width': '75%', 'height': '800px'} #'border':'2px black solid', 'borderRadius':'20px', 'float': 'right'
                ),
                html.Div([
                    html.Div([
                        html.P(id= 'mouse-click'),
                    ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'display': 'inline-block', 'padding' : '5px 5px 5px 5px', 'width': '100%'}),

                    dcc.Checklist(options=['Show weights'], id='checklist-show', value=['Show weights'], style={'margin-top':'10px', 'margin-bottom':'10px'}),

                    dcc.Upload(id='upload-file', children=html.Div(['Drag and Drop or ', html.A('Select a File')]),style={'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px', 'textAlign': 'center', 'width': '100%',},),

                    html.Div(id='output-upload-file'),
                ], style={'vertical-align': 'top', 'display': 'inline-block', 'width': '10%',"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'padding' : '10px 10px 10px 10px'}),
                ]),
            ],type='circle',
        ),
    dcc.Store(id='loaded-graph'),
    dcc.Store(id='gen-degrees'),

], style={'margin-left' : '10px', 'margin-right' : '10px', 'margin-top' : '10px', 'margin-bottom' : '10px'})


@app.callback(Output('dropdown-sample', 'options'),                     # disable sample dropdown if allCelltypeGraph or only one sample in celltype
              Output('dropdown-sample' ,'value'),
              Output('gen-degrees', 'data'),
              Input('dropdown-graph', 'value'))
def update_file(celltype):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    gr = gt.load_graph(f"../data/graph-tool/{celltype}.gt")
    gen_degrees = {gr.vp.name[v] : len(gr.get_in_edges(v)) for v in gr.vertices() if gr.vp.name[v].startswith("E")}
    sample_list = [s for s in gr.edge_properties if s not in ("celltype", "celltypeID", "rem")]
    val_sample = sample_list[0]
    return sample_list, val_sample, gen_degrees

@app.callback(Output('dropdown-gen', 'options'),            # dauert zu lange, vllt sideeffect, aber eig überprüft dass kein sideeffect
              Output('dropdown-gen', 'value'),
              Input('gen-degrees', 'data'),
              Input('dropdown-graph', 'value'),
              Input('filter-gen', 'value'))
def update_gen_dropdown(gen_degrees, celltype, gen_filter):
    if ctx.triggered_id == 'dropdown-graph':
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen
    elif ctx.triggered_id == 'filter-gen' and gen_filter is not None:
        gen_list = [k for k, v in gen_degrees.items() if v >= gen_filter]
        return gen_list, gen_list[0]
    else:
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen


@app.callback(Output('network-graph', 'elements'),
              Output('loaded-graph', 'data'),
              Output('output-upload-file', 'children'),
              Output('filter-weight-up', 'value'),
              Output('filter-weight-low', 'value'),
              Output('checklist-show', 'value'),

              Input('dropdown-gen', 'value'),
              Input('input-rem', 'value'),
              Input('input-crem', 'value'),
              Input('dropdown-sample', 'value'),
              Input('upload-file', 'contents'),

              State('network-graph', 'elements'),
              State('upload-file', 'filename'),
              State('output-upload-file', 'children'))
def update_elements(gen, rem_int, crem_int, sample, contents, elements, filename, out_file):
    if ctx.triggered_id == 'upload-file':
        if contents is not None and '.csv' in filename:
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            try:
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                net = get_network(gen, sample, True, df)
                return net, net, html.Div(['File uploaded successfully.']), None, None, ["Show weights"]
            except Exception as e:
                return elements, elements, html.Div([str(e)]), None, None, ["Show weights"]
        else: return elements, elements, html.Div(["Please provide a csv file in the correct format."]), None, None, ["Show weights"]
    if ctx.triggered_id == 'dropdown-gen':
        net = get_network(gen, sample)
        return net, net, out_file, None, None, ["Show weights"]
    elif ctx.triggered_id == 'input-rem':
        rem = f'REM{rem_int:>07}'
        net = get_network(rem, sample)
        return net, net, out_file, None, None, ["Show weights"]
    elif ctx.triggered_id == 'input-crem':
        crem = f'CREM{crem_int:>07}'
        net = get_network(crem, sample)
        return net, net, out_file, None, None, ["Show weights"]
    else:
        net = get_network(gen, sample)
        return net, net, out_file, None, None, ["Show weights"]

@app.callback(Output('network-graph', 'elements', allow_duplicate=True),
              Input('filter-weight-up', 'value'),
              Input('filter-weight-low', 'value'),
              Input('checklist-show', 'value'),
              State('network-graph', 'elements'),
              State('loaded-graph', 'data'),
              prevent_initial_call=True)
def filter_elements(filter_up, filter_low, show, elements, loaded_graph):
    return filter_network(loaded_graph, filter_up, filter_low, show)

@app.callback(Output('filter-weight-up', 'value', allow_duplicate=True),
              Output('filter-weight-low', 'value', allow_duplicate=True),
              Output('filter-gen', 'value'),
              Input('reset-filter', 'n_clicks'),
              prevent_initial_call=True)
def reset_filter(n_clicks):
    return None, None, None


@app.callback(Output('mouse-click', 'children'),
              Input('network-graph', 'tapNodeData'))
def node_hover(data):
    if data:
        if len(data) == 6:
            return html.Strong('Node info'), html.Br(), f'Name: {data["label"]}', html.Br(), f'Expression: {data["expr"]}', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f'Start: {data["start"]} ', html.Br(),f'End: {data["end"]}'
        else:
            return html.Strong('Node info'), html.Br(), f'Name: {data["label"]} ', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f' Start: {data["start"]} ', html.Br(),f' End: {data["end"]}'
    else:
        return html.Strong('Node info'), html.Br()


# app.run_server(host='0.0.0.0', port=8050, debug=False)
app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [5]:
# import matplotlib.pyplot as plt
#
# weights = [g.ep.standDnase1Log2[e] for e in g.edges()]
# plt.hist(weights)
# plt.xlabel('weight')
# plt.show()

In [6]:
# graph_pickle_safe = open("graph_pickle", "wb")
# gen_list = [{'label': name, 'value': name} for name in [a for a in list(gr.vp.name) if a.startswith("E")]]
# pickle.dump(gr, graph_pickle_safe)
# graph_pickle_safe.close()

In [7]:
# g = gt.load_graph('../data/graph-tool/allCelltypeGraph.gt')
# v = gu.find_vertex(g, g.vp.name, 'ENSG00000278267')[0]
# cyto_list = []
# graph_type = 'allCelltypeGraph.gt'
# nodes = []
# edges= []
# # vfilt = g.new_vertex_property('bool')
# # vfilt[v] = True
# nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v]}, 'classes':'green'})
# for e, w in zip(v.in_edges(), v.in_neighbors()):
#     # vfilt[w] = True
#     nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w]}, 'classes': 'red triangle'})
#     if graph_type != "allCelltypeGraph.gt":
#         edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.standDnase1Log2[e]}})
#     else:
#         if g.ep.score[e] >= 0: color = "green"
#         else: color = "red"
#         edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.score[e]}, 'style':{'width':g.ep.score[e]*5, 'color': color}})
# # sub = GraphView(g, vfilt)
# cyto_list = nodes+edges
# cyto_list